In [1]:
# start a pyspark session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('google_playstore_analysis').getOrCreate()

In [2]:
spark

In [3]:
## Read the data from the csv file into a DataFrame.
df_pyspark = spark.read.csv('googleplaystore.csv', header=True, inferSchema=True)
## Replacing NaN with null values 
df_pyspark = df_pyspark.replace('NaN', None)

In [4]:
df_pyspark.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



# DATA CLEANING

In [5]:
#checking for the null values in the rating column as rating data type should be double
from pyspark.sql.functions import col
df_pyspark.filter(col("Rating").isNull()).count()

1474

In [6]:
df_pyspark.describe("Rating").show()

+-------+------------------+
|summary|            Rating|
+-------+------------------+
|  count|              9367|
|   mean| 4.193315536572349|
| stddev|0.5374844456576829|
|    min|        navigation|
|    max|              Body|
+-------+------------------+



In [8]:
from pyspark.sql.functions import col,when,regexp_replace

non_numeric_rating = df_pyspark.filter(~col("Rating").rlike("^[0-9]*\\.?[0-9]+$"))
non_numeric_rating.count()

2

In [9]:
## replacing non-numeric values to nulls and converting entire column to double
df_cleaned = df_pyspark.withColumn(
    "Rating",
    when(col("Rating").rlike("^[0-9]*\\.?[0-9]+$"), col("Rating").cast("double")).otherwise(None)
)

In [10]:
df_cleaned.describe("Rating").show()

+-------+------------------+
|summary|            Rating|
+-------+------------------+
|  count|              9365|
|   mean| 4.193315536572349|
| stddev|0.5374844456576829|
|    min|               1.0|
|    max|              19.0|
+-------+------------------+



In [11]:
df_cleaned.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



reviews should be in INT but from the printSchema we can see that it is String

In [12]:
df_pyspark.describe("Reviews").show()

+-------+-----------------+
|summary|          Reviews|
+-------+-----------------+
|  count|            10841|
|   mean|444225.1924709356|
| stddev|2928025.838407947|
|    min|          camera"|
|    max|    weight lose)"|
+-------+-----------------+



In [13]:
## find out how many columns are not null and numeric 
non_numeric_reviews = df_cleaned.filter(~col("Reviews").rlike("^[0-9]+$"))
non_numeric_reviews.count()

3

I want to replace these with null values

In [18]:
df_cleaned = df_cleaned.withColumn(
    "Reviews",
    when(col("Reviews").rlike("^[0-9]+$"), col("Reviews").cast("int")).otherwise(None))

In [19]:
df_cleaned.describe("Reviews").show()

+-------+-----------------+
|summary|          Reviews|
+-------+-----------------+
|  count|            10838|
|   mean|444225.1924709356|
| stddev|2928025.838407947|
|    min|                0|
|    max|         78158306|
+-------+-----------------+



In [16]:
df_cleaned.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



remove + from the Installs column and change data type to Integer

In [20]:
df_cleaned=df_cleaned.withColumn("Installs",regexp_replace(col("Installs"), "[+,]", ""))

In [21]:
df_cleaned=df_cleaned.withColumn("Installs",col("Installs").cast("int"))

In [22]:
df_cleaned.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [23]:
df_cleaned.describe("Price").show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|             10841|
|   mean|               0.0|
| stddev|               0.0|
|    min|             $0.99|
|    max|Varies with device|
+-------+------------------+



In [24]:
## remove dollar sign from the price column
df_cleaned=df_cleaned.withColumn("Price",regexp_replace(col("Price"),"[$,M]",""))

In [25]:
df_cleaned.describe("Price").show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|             10841|
|   mean|  1.02792416274563|
| stddev| 15.95047880645482|
|    min|                 0|
|    max|Varies with device|
+-------+------------------+



In [26]:
non_numeric_Price = df_cleaned.filter(~col("Price").rlike("^[0-9]*\\.?[0-9]+$"))
non_numeric_Price.show()

+--------------------+-------------+------+-------+-------------------+--------+------+------------------+--------------+-----------------+------------+-----------+-----------------+
|                 App|     Category|Rating|Reviews|               Size|Installs|  Type|             Price|Content Rating|           Genres|Last Updated|Current Ver|      Android Ver|
+--------------------+-------------+------+-------+-------------------+--------+------+------------------+--------------+-----------------+------------+-----------+-----------------+
|"Yanosik: ""antyr...| traffic jams|  NULL|   NULL|MAPS_AND_NAVIGATION|       4|102248|Varies with device|    5,000,000+|             Free|           0|   Everyone|Maps & Navigation|
|Life Made WI-Fi T...|          1.9|  19.0|   NULL|             1,000+|    NULL|     0|          Everyone|          NULL|February 11, 2018|      1.0.19| 4.0 and up|             NULL|
+--------------------+-------------+------+-------+-------------------+--------+-----

replace with 0 where price data cells are in non numeric format

In [27]:
df_cleaned.describe("Price").show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|             10841|
|   mean|  1.02792416274563|
| stddev| 15.95047880645482|
|    min|                 0|
|    max|Varies with device|
+-------+------------------+



 find out TOP 10 reviews given to the apps 

In [33]:
df_cleaned.createOrReplaceTempView("google_playstore")

In [49]:
top_reviews_sql = spark.sql("""
    SELECT App, SUM(Reviews) AS TotalReviews
    FROM google_playstore
    GROUP BY App
    ORDER BY TotalReviews DESC
    LIMIT 10
""")


In [50]:
top_reviews_sql.show()

+--------------------+------------+
|                 App|TotalReviews|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+



TOP 10 INSTALL APPS

In [55]:
top_install_apps = spark.sql("""
SELECT App,Type,SUM(Installs) as TotalInstalls
FROM google_playstore
GROUP BY 1,2
order by 3 desc 
LIMIT 10
""")
top_install_apps.show()

+------------------+----+-------------+
|               App|Type|TotalInstalls|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+



CATEGORYWISE_INSTALLATIONS

In [62]:
categorywise_Installations = spark.sql("""
SELECT Category, SUM(Installs) as TotalInstalls
FROM google_playstore
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
categorywise_Installations.show()

+------------------+-------------+
|          Category|TotalInstalls|
+------------------+-------------+
|              GAME|  35086024415|
|     COMMUNICATION|  32647276251|
|      PRODUCTIVITY|  14176091369|
|            SOCIAL|  14069867902|
|             TOOLS|  11452771915|
|            FAMILY|  10258263505|
|       PHOTOGRAPHY|  10088247655|
|NEWS_AND_MAGAZINES|   7496317760|
|  TRAVEL_AND_LOCAL|   6868887146|
|     VIDEO_PLAYERS|   6222002720|
+------------------+-------------+



TOP PAID APPS

In [64]:
top_paid_apps = spark.sql("""
SELECT App, SUM(Price) as TotalPrice
FROM google_playstore
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
top_paid_apps.show()

+--------------------+----------+
|                 App|TotalPrice|
+--------------------+----------+
|I'm Rich - Trump ...|     400.0|
|most expensive ap...|    399.99|
|  I am rich(premium)|    399.99|
|   I Am Rich Premium|    399.99|
|      I am Rich Plus|    399.99|
|I'm Rich/Eu sou R...|    399.99|
|       I Am Rich Pro|    399.99|
|  I AM RICH PRO PLUS|    399.99|
|           I am Rich|    399.99|
|          I am Rich!|    399.99|
+--------------------+----------+



TOP PAID RATING APPS

In [69]:
top_paid_rating_apps = spark.sql("""
SELECT App,SUM(Price) 
FROM google_playstore
WHERE Type = "Paid"
GROUP by 1
ORDER BY 2 DESC
LIMIT 10
""")
top_paid_rating_apps.show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|     400.0|
|most expensive ap...|    399.99|
|  I am rich(premium)|    399.99|
|   I Am Rich Premium|    399.99|
|      I am Rich Plus|    399.99|
|I'm Rich/Eu sou R...|    399.99|
|       I Am Rich Pro|    399.99|
|  I AM RICH PRO PLUS|    399.99|
|           I am Rich|    399.99|
|          I am Rich!|    399.99|
+--------------------+----------+

